# Week 3 - Sensitivity analysis

This exercise uses the same predator-prey model we used for the multi-model exercise, focusing on the Python version. As with the other exercise, define a model object for the function below, with the uncertainty ranges provided:

|Parameter	|Range or value	        |
|-----------|--------------:|
|prey_birth_rate    	|0.015 – 0.035	|
|predation_rate|0.0005 – 0.003 	|
|predator_efficiency     	|0.001 – 0.004	    |
|predator_loss_rate	    |0.04 – 0.08	    |

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from ema_workbench import (Model, RealParameter, TimeSeriesOutcome, perform_experiments, ema_logging, ScalarOutcome)

from ema_workbench import Samplers

from ema_workbench.analysis import feature_scoring
from ema_workbench.analysis.scenario_discovery_util import RuleInductionType
from ema_workbench.em_framework.salib_samplers import get_SALib_problem
from SALib.analyze import sobol

C:\Users\muham\OneDrive - Delft University of Technology\Documents\Kuliah\S2\Q1.4\Model-based Decision Making\Assignments\venv\Lib\site-packages\ema_workbench\em_framework\__init__.py:101: UserWarning: ipyparallel not installed - IpyparalleEvaluator not available
  warnings.warn("ipyparallel not installed - IpyparalleEvaluator not available")


In [2]:
def pred_prey(prey_birth_rate=0.025, predation_rate=0.0015, predator_efficiency=0.002,
             predator_loss_rate=0.06, initial_prey=50, initial_predators=20, dt=0.25,
             final_time=365, reps=1):

    #Initial values
    predators = np.zeros((reps, int(final_time/dt)+1))
    prey = np.zeros((reps, int(final_time/dt)+1))
    sim_time = np.zeros((reps, int(final_time/dt)+1))

    for r in range(reps):

        predators[r,0] = initial_predators
        prey[r,0] = initial_prey

    #Calculate the time series
    for t in range(0, sim_time.shape[1]-1):

        dx = (prey_birth_rate*prey[r,t]) - (predation_rate*prey[r,t]*predators[r,t])
        dy = (predator_efficiency*predators[r,t]*prey[r,t]) - (predator_loss_rate*predators[r,t])

        prey[r,t+1] = max(prey[r,t] + dx*dt, 0)
        predators[r,t+1] = max(predators[r,t] + dy*dt, 0)
        sim_time[r,t+1] = (t+1)*dt

    #Return outcomes
    return {'TIME':sim_time,
            'predators':predators,
            'prey':prey}

In [3]:
# Instantiate Python Model
model = Model('PredPreyModel', function=pred_prey)


# Define uncertainties and outcomes
model.uncertainties = [
    RealParameter('prey_birth_rate', 0.015, 0.035),
    RealParameter('predation_rate', 0.0005, 0.003),
    RealParameter('predator_efficiency', 0.001, 0.004),
    RealParameter('predator_loss_rate', 0.04, 0.08)
]

model.outcomes = [
    TimeSeriesOutcome('TIME'),
    TimeSeriesOutcome('predators'),
    TimeSeriesOutcome('prey')
]



In [4]:
results = perform_experiments(model, 50)

100%|█████████████████████████████████████████| 50/50 [00:00<00:00, 510.28it/s]


In [10]:
experiments, outcomes = results

In [14]:
outcomes

{'TIME': array([[[0.0000e+00, 2.5000e-01, 5.0000e-01, ..., 3.6450e+02,
          3.6475e+02, 3.6500e+02]],
 
        [[0.0000e+00, 2.5000e-01, 5.0000e-01, ..., 3.6450e+02,
          3.6475e+02, 3.6500e+02]],
 
        [[0.0000e+00, 2.5000e-01, 5.0000e-01, ..., 3.6450e+02,
          3.6475e+02, 3.6500e+02]],
 
        ...,
 
        [[0.0000e+00, 2.5000e-01, 5.0000e-01, ..., 3.6450e+02,
          3.6475e+02, 3.6500e+02]],
 
        [[0.0000e+00, 2.5000e-01, 5.0000e-01, ..., 3.6450e+02,
          3.6475e+02, 3.6500e+02]],
 
        [[0.0000e+00, 2.5000e-01, 5.0000e-01, ..., 3.6450e+02,
          3.6475e+02, 3.6500e+02]]], shape=(50, 1, 1461)),
 'predators': array([[[20.        , 20.6830627 , 21.38369618, ...,  9.4464097 ,
           9.32008436,  9.19544088]],
 
        [[20.        , 20.48012349, 20.964019  , ..., 23.38505181,
          23.96208313, 24.5415209 ]],
 
        [[20.        , 20.11982834, 20.23653714, ..., 23.0491076 ,
          23.09512524, 23.13670369]],
 
        ...,
 
 

In [19]:
experiments

,predation_rate,predator_efficiency,predator_loss_rate,prey_birth_rate,scenario,policy,model
0,0.001956,0.003928,0.059789,0.016440,0,None,PredPreyModel
1,0.002925,0.003480,0.077988,0.023692,1,None,PredPreyModel
2,0.002818,0.001791,0.065571,0.022274,2,None,PredPreyModel
3,0.000590,0.001000,0.073049,0.034026,3,None,PredPreyModel
4,0.001568,0.001934,0.062261,0.022719,4,None,PredPreyModel
5,0.000831,0.002668,0.047266,0.021167,5,None,PredPreyModel
6,0.002389,0.001331,0.075134,0.017281,6,None,PredPreyModel
7,0.001152,0.001278,0.068431,0.030556,7,None,PredPreyModel
8,0.002733,0.001168,0.057135,0.027319,8,None,PredPreyModel
9,0.000696,0.003132,0.047124,0.028080,9,None,PredPreyModel


#### 1. Sensitivity analysis
Sensitivity analysis often focuses on the final values of an outcome at the end of the simulation. However, we can also look at metrics that give us additional information about the behavior of the model over time. Using [the statsmodel library](https://www.statsmodels.org/stable/index.html) and an appropriate sampling design, fit a linear regression model for each of the following indicators. What can we conclude about the behavior of the model, and about the importance of the different inputs?

  * The final values of the _prey_ outcome
  * The mean values of the _prey_ outcome over time, within each experiment
  * The standard deviations of the _prey_ outcome over time, within each experiment

In [20]:
import numpy as np
import pandas as pd

# Access only the prey outcome
prey = outcomes['prey']  # shape: (n_experiments, 1, n_timesteps)

# Remove singleton dimension: (n_experiments, n_timesteps)
prey = prey[:, 0, :]

# Compute metrics
final_values = prey[:, -1]
mean_values = prey.mean(axis=1)
std_dev_values = prey.std(axis=1)

# Combine into a DataFrame
df_metrics = pd.DataFrame({
    'final_prey': final_values,
    'mean_prey': mean_values,
    'std_prey': std_dev_values
})
df_metrics

,final_prey,mean_prey,std_prey
0,1.601754,14.790064,15.708095
1,49.619086,22.718363,17.372717
2,40.203379,36.920840,18.786499
3,31.694123,77.236727,37.973442
4,38.772588,32.841200,12.139809
5,5.075325,16.458262,15.685775
6,76.423169,56.434012,24.504178
7,46.747651,54.656196,7.431394
8,25.910769,47.635671,18.950478
9,5.199434,13.509655,16.400103


In [22]:
# Keep only the columns needed for sensitivity analysis
df_inputs = experiments[['predation_rate', 'predator_efficiency', 'predator_loss_rate', 'prey_birth_rate']].copy()


In [23]:
import statsmodels.api as sm

# Add constant for intercept
X = sm.add_constant(df_inputs)

# Dictionary to store models
models = {}

for target in ['final_prey', 'mean_prey', 'std_prey']:
    y = df_metrics[target]
    model = sm.OLS(y, X).fit()
    models[target] = model
    print(f"\n📈 Regression summary for {target}:\n")
    print(model.summary())


📈 Regression summary for final_prey:

                            OLS Regression Results                            
Dep. Variable:             final_prey   R-squared:                       0.445
Model:                            OLS   Adj. R-squared:                  0.396
Method:                 Least Squares   F-statistic:                     9.026
Date:                Wed, 14 May 2025   Prob (F-statistic):           1.93e-05
Time:                        10:56:00   Log-Likelihood:                -207.17
No. Observations:                  50   AIC:                             424.3
Df Residuals:                      45   BIC:                             433.9
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------

#### 2. SOBOL
Use the Sobol sampling functionality included in the Workbench to perform experiments with a sample size of N=50, then analyze the results with SALib for the same three indicators. This requires specifying the keyword argument `'uncertainty_sampling'` of perform_experiments. Note that when using Sobol sampling, the meaning of the keyword argument `scenarios` changes a bit. In order to properly estimate Sobol scores as well as interaction effects, you require N * (2D+2) scenarios, where D is the number of uncertain parameters, and N is the value for scenarios passed to `perform_experiments`. Repeat the analysis for larger sample sizes, with N=250 and N=1000. How can we interpret the first-order and total indices? Are these sample sizes sufficient for a stable estimation of the indices? You'll need to use the [get_SALib_problem](https://emaworkbench.readthedocs.io/en/latest/ema_documentation/em_framework/salib_samplers.html) function to convert your Workbench experiments to a problem definition that you can pass to the SALib analysis function.

*hint*: sobol is a deterministic sequence of quasi random numbers. Thus, you can run with N=1000 and simply use slicing to get the results for N=50 and N=250.

#### 3. Extra trees
Use the [Extra-Trees analysis](https://emaworkbench.readthedocs.io/en/latest/ema_documentation/analysis/feature_scoring.html) included in the Workbench to approximate the Sobol total indices, with a suitable sampling design. As a starting point, use an ensemble of 100 trees and a max_features parameter of 0.6, and set the analysis to regression mode. Are the estimated importances stable relative to the sample size and the analysis parameters? How do the results compare to the Sobol indices? For more details on this analysis see [Jaxa-Rozen & Kwakkel (2018)](https://www.sciencedirect.com/science/article/pii/S1364815217311581)